In [18]:
import numpy as np



In [180]:
class neural_network:
    def __init__(self,learning_rate,epochs,output_size,hidden_layer_size,k1,k2,x):
        self.learning_rate=learning_rate
        self.epochs=epochs
        self.output_size=output_size
        self.hidden_layer_size=hidden_layer_size
        self.k1=k1
        self.k2=k2
        self.x=x
        kh,kw=3,3
        N,heightinp1,widthinp1,channelsinp1=self.x.shape
        self.kernel1=np.random.randn(kh,kw,channelsinp1,self.k1)*0.01
        self.bias1=np.zeros(self.k1)
        self.kernel2=np.random.randn(kh,kw,self.k1,self.k2)*0.01
        self.bias2=np.zeros(self.k2)
        s=1
        conv2_hout=((heightinp1-2*kh)//s)+2
        conv2_wout=((widthinp1-2*kw)//s)+2
        flattened=(N,conv2_hout*conv2_wout*self.k2)
        self.w1=np.random.randn(flattened[1],self.hidden_layer_size)*0.01
        self.b1=np.zeros((1,self.hidden_layer_size))
        self.w2=np.random.randn(self.hidden_layer_size,self.output_size)*0.01
        self.b2=np.zeros((1,self.output_size))
        self.accuracy_history=[]
        self.loss_history=[]
    def relu(self,z):
        return np.maximum(z,0)
    def reluderivative(self,z):
        return np.where(z>0,1,0)
    def softmax(self,z):
        z=z-np.max(z, axis=1,keepdims=True)
        expz=np.exp(z)
        return expz/np.sum(expz,axis=1,keepdims=True)
    def accuracy(self,y,y_pred):
        y_pre=np.argmax(y_pred,axis=1)
        y_true=np.argmax(y,axis=1)
        return np.mean(y_pre==y_true)
    def loss(self, y_pred, y):
        eps = 1e-9
        y_pred = np.clip(y_pred,eps,1-eps)
        return -np.mean(np.sum(y*np.log(y_pred),axis=1))

    def forward_conv(self,s):
        N,heightinp1,widthinp1,channelsinp1=self.x.shape
        k1h,k1w,k1c,k1n=self.kernel1.shape
        k2h,k2w,k2c,k2n=self.kernel2.shape
        h_out1=((heightinp1-k1h)//s)+1
        w_out1=((widthinp1-k1w)//s)+1
        x_input1=np.zeros((N,h_out1*w_out1,k1h*k1w*k1c))
        self.conv1op1=np.zeros((N,h_out1*w_out1,k1n))
        for n in range(N):
            index=0
            for i in range(0,h_out1,s):
                for j in range(0,w_out1,s):
                    patch=self.x[n,i:i+k1h,j:j+k1w,:]
                    x_input1[n,index,:]=patch.flatten()
                    index+=1
            self.conv1op1[n]=np.dot(x_input1[n],self.kernel1.reshape(-1,k1n))+self.bias1.reshape(-1,k1n)
        self.conv1op1=self.conv1op1.reshape(N,h_out1,w_out1,k1n)
        self.relu_output1=self.relu(self.conv1op1)
        _,heightinp1,widthinp1,channeslinp1=self.relu_output1.shape
        h_out1=((heightinp1-k2h)//s)+1
        w_out1=((widthinp1-k2w)//s)+1
        
        x_input2=np.zeros((N,h_out1*w_out1,k2h*k2w*k1n))
        self.conv2op1=np.zeros((N,h_out1*w_out1,k2n))
        for n in range(N):
            index=0
            for i in range(0,h_out1,s):
                for j in range(0,w_out1,s):
                    patch=self.relu_output1[n,i:i+k2h,j:j+k2w,:]
                    x_input2[n,index,:]=patch.flatten()
                    index+=1
            self.conv2op1[n]=np.dot(x_input2[n],self.kernel2.reshape(-1,k2n))+self.bias2.reshape(-1,k2n)
        self.conv2op1=self.conv2op1.reshape(N,h_out1,w_out1,k2n)
        self.relu_output2=self.relu(self.conv2op1)
        flatted=self.relu_output2.reshape(N,-1)
        self.z1=np.dot(flatted,self.w1)+self.b1
        self.r1=self.relu(self.z1)
        self.z2=np.dot(self.r1,self.w2)+self.b2
        y_pred=self.softmax(self.z2)
        return y_pred,flatted
    def backpropagation(self,y,y_pred,flatted,s=1):
        k1h,k1w,k1c,k1n=self.kernel1.shape
        k2h,k2w,k2c,k2n=self.kernel2.shape
        N,h_out1,w_out1,chanels=self.relu_output2.shape
        _,h_out2,w_out2,_=self.relu_output1.shape
        
        error=y_pred-y
        
        gradientsw2=np.dot(self.r1.T,error)/len(y)
        gradientsb2=np.sum(error,axis=0)/len(y)

        errorz1=np.dot(error,self.w2.T)*self.reluderivative(self.z1)
        
        gradientsw1=np.dot(flatted.T,errorz1)/len(y)
        gradientsb1=np.sum(errorz1,axis=0)/len(y)

        gradientsreluop1=np.dot(errorz1,self.w1.T)
        gradientsreluop1=gradientsreluop1.reshape(N,h_out1,w_out1,k2n)
        convdelta2=gradientsreluop1*self.reluderivative(self.conv2op1)
        convdelta2=convdelta2.reshape(N,h_out1*w_out1,k2n)
        relu1reshape=np.zeros((N,h_out1*w_out1,k2h*k2w*k2c))
        for n in range(N):
            index=0
            for i in range(0,h_out1,s):
                for j in range(0,w_out1,s):
                    patch=self.relu_output1[n,i:i+k2h,j:j+k2w,:]
                    relu1reshape[n,index,:]=patch.flatten()
                    index+=1
        gradientsk2=np.einsum('npi,npo->io',relu1reshape,convdelta2)
        gradientsk2=gradientsk2.reshape(k2h,k2w,k2c,k2n)
        gradientsb22=np.sum(convdelta2,axis=(0,1))

        pad=k1h-1
        convdelta2=convdelta2.reshape(N,h_out1,w_out1,k2n)

        error_pad=np.pad(convdelta2,pad_width=((0,0,),(pad,pad),(pad,pad),(0,0)),mode='constant')
        flipped_kernel=self.kernel2[::-1,::-1,:,:]
        flipped_kernel=np.transpose(flipped_kernel,(0,1,3,2))
        flipped_kernel=flipped_kernel.reshape(-1,k2c)
        
        
        temprelu=np.zeros((N,h_out2*w_out2,k2h*k2w*k2n))
        conv11op1=np.zeros((N,h_out2*w_out2,k2c))
        
        for n in range(N):
            index=0
            for i in range(0,h_out2,s):
                for j in range(0,w_out2,s):
                    patch=error_pad[n,i:i+k2h,j:j+k2w,:]
                    temprelu[n,index,:]=patch.flatten()
                    index+=1
            conv11op1[n]=np.dot(temprelu[n],flipped_kernel)
        conv11op1=conv11op1.reshape(N,h_out2,w_out2,k2c)
        conv11op1=conv11op1*self.reluderivative(self.conv1op1)

        x_col=np.zeros((N,h_out2*w_out2,k1h*k1w*k1c))
        
        for n in range(N):
            index=0
            for i in range(0,h_out2,s):
                for j in range(0,w_out2,s):
                    patch=self.x[n,i:i+k1h,j:j+k1w,:]
                    x_col[n,index,:]=patch.flatten()
            conv11op1=conv11op1.reshape(N,h_out2*w_out2,k2c)   
        gradientsk1=np.einsum('npi,npo->io',x_col,conv11op1)
        gradientsk1=gradientsk1.reshape(k1h,k1w,k1c,k1n)
        gradientsb11=np.sum(conv11op1,axis=(0,1))


        self.w2-=self.learning_rate*gradientsw2
        self.b2-=self.learning_rate*gradientsb2
        self.w1-=self.learning_rate*gradientsw1
        self.b1-=self.learning_rate*gradientsb1
        self.kernel2-=self.learning_rate*gradientsk2
        self.bias2-=self.learning_rate*gradientsb22
        self.kernel1-=self.learning_rate*gradientsk1
        self.bias1-=self.learning_rate*gradientsb11
    def training(self,x,y):
        for epoch in range(self.epochs):
            y_pred,flatted=self.forward_conv(s=1)
            acc=self.accuracy(y_pred,y)
            self.accuracy_history.append(acc)
            los=self.loss(y_pred,y)
            self.loss_history.append(los)
            self.backpropagation(y,y_pred,flatted,1)
            if epoch%10==0:
                print(f'epoch number:{epoch} and accuracy:{self.accuracy_history[epoch]} loss:{self.loss_history[epoch]}')
            
          

import tensorflow as tf
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

from sklearn.preprocessing import OneHotEncoder
en=OneHotEncoder(sparse_output=True)
y_train=y_train.reshape(-1,1)
y=en.fit_transform(y_train).toarray()


y=y[:100]
x=x_train[:100]

In [182]:
x[1].shape

(28, 28, 1)

In [183]:
x=x.reshape(-1,28,28,1)

In [184]:
x.shape

(100, 28, 28, 1)

In [185]:
cnn=neural_network(0.001,100,10,10,3,4,x)

In [186]:
cnn.training(x,y)

epoch number:0 and accuracy0.07 loss2.3024980315355617
epoch number:10 and accuracy0.13 loss2.3023790749156903
epoch number:20 and accuracy0.17 loss2.3022614870406417
epoch number:30 and accuracy0.15 loss2.302143379814563
epoch number:40 and accuracy0.15 loss2.302021105470434
epoch number:50 and accuracy0.15 loss2.3018971159436346
epoch number:60 and accuracy0.16 loss2.3017706607492197
epoch number:70 and accuracy0.16 loss2.3016411908612535
epoch number:80 and accuracy0.16 loss2.3015068601393565
epoch number:90 and accuracy0.16 loss2.3013640802386277


In [3]:
import numpy as np
k=np.random.randn(3,3,2,2)

In [10]:
k

array([[[[-9.24232031e-01,  7.06010213e-01],
         [ 2.92739194e-01, -6.54256192e-01]],

        [[-7.42585064e-01,  1.29728310e+00],
         [ 2.23668465e-02,  1.51621625e+00]],

        [[-1.18307456e-01,  2.66498380e-01],
         [ 7.85470442e-01,  1.62879470e+00]]],


       [[[-7.30941873e-01, -1.05208280e+00],
         [ 6.48386943e-02, -6.13370427e-04]],

        [[ 1.11424546e+00,  9.30095427e-01],
         [-9.83481872e-02, -2.60299815e-01]],

        [[ 4.74490066e-01,  6.20898736e-01],
         [ 1.11917893e-02,  8.20782191e-02]]],


       [[[-5.41277064e-02,  1.87810951e+00],
         [-1.68868570e+00, -4.50950731e-01]],

        [[ 8.26093906e-01, -5.85645135e-01],
         [-6.27042756e-01,  5.32229719e-02]],

        [[-1.27135944e+00,  1.12147565e+00],
         [ 1.51815733e-01, -3.94286686e-01]]]])

In [11]:
k1=np.transpose(k,(0,1,3,2))
k1

array([[[[-9.24232031e-01,  2.92739194e-01],
         [ 7.06010213e-01, -6.54256192e-01]],

        [[-7.42585064e-01,  2.23668465e-02],
         [ 1.29728310e+00,  1.51621625e+00]],

        [[-1.18307456e-01,  7.85470442e-01],
         [ 2.66498380e-01,  1.62879470e+00]]],


       [[[-7.30941873e-01,  6.48386943e-02],
         [-1.05208280e+00, -6.13370427e-04]],

        [[ 1.11424546e+00, -9.83481872e-02],
         [ 9.30095427e-01, -2.60299815e-01]],

        [[ 4.74490066e-01,  1.11917893e-02],
         [ 6.20898736e-01,  8.20782191e-02]]],


       [[[-5.41277064e-02, -1.68868570e+00],
         [ 1.87810951e+00, -4.50950731e-01]],

        [[ 8.26093906e-01, -6.27042756e-01],
         [-5.85645135e-01,  5.32229719e-02]],

        [[-1.27135944e+00,  1.51815733e-01],
         [ 1.12147565e+00, -3.94286686e-01]]]])